In [29]:
import json 
import notebookutils 
import sempy.fabric as fabric 
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException
import pandas as pd

StatementMeta(, 13261b34-66d2-40be-9a6d-b15e5a50d9f1, 31, Finished, Available, Finished)

In [30]:
# Semantic Link Client
client = fabric.FabricRestClient()

StatementMeta(, 13261b34-66d2-40be-9a6d-b15e5a50d9f1, 32, Finished, Available, Finished)

### Parameters

In [ ]:
# current workspace
workspace_id=spark.conf.get("trident.workspace.id")

# explicit Lakehouse Name (you can use se your Parameter here)
lakehouse_name = "LAKEHOUSE_NAME" # <-------------------------------- CHANGE THIS

StatementMeta(, 13261b34-66d2-40be-9a6d-b15e5a50d9f1, 33, Finished, Available, Finished)

### Function to find the Lakehouse-ID by name

In [32]:
def find_lakehouse_id(client: client, workspace_id: str, lakehouse_name: str) -> str | None:
    items = client.get_paged(f"/v1/workspaces/{workspace_id}/items")
    df = pd.json_normalize(items)
    df_match = df[(df['displayName'] == lakehouse_name) & (df['type'].str.lower() == 'lakehouse')]
    if not df_match.empty:
        return df_match.iloc[0]['id']
    return None

StatementMeta(, 13261b34-66d2-40be-9a6d-b15e5a50d9f1, 34, Finished, Available, Finished)

### Helper function

In [ ]:
def pad_or_truncate_string(input_string, length, pad_char=' '):
    # Truncate if the string is longer than the specified length
    if len(input_string) > length:
        return input_string[:length]
    # Pad if the string is shorter than the specified length
    return input_string.ljust(length, pad_char)

StatementMeta(, 13261b34-66d2-40be-9a6d-b15e5a50d9f1, 35, Finished, Available, Finished)

### MDSyncNewRESTAPI

In [ ]:
#workspace_id=spark.conf.get("trident.workspace.id")
#lakehouse_id=spark.conf.get("trident.lakehouse.id")

#Instantiate the client
#client = fabric.FabricRestClient()

#find lakehouse ID
lakehouse_id = find_lakehouse_id(client, workspace_id, lakehouse_name)

# This is the SQL endpoint I want to sync with the lakehouse, this needs to be the GUI
sqlendpoint = fabric.FabricRestClient().get(f"/v1/workspaces/{workspace_id}/lakehouses/{lakehouse_id}").json()['properties']['sqlEndpointProperties']['id']

# URI for the call 
#uri = f"v1/workspaces/{workspace_id}/sqlEndpoints/{sqlendpoint}/refreshMetadata?preview=true" #preview=true is no longer needed, the result is slightly different with preview=true
uri = f"v1/workspaces/{workspace_id}/sqlEndpoints/{sqlendpoint}/refreshMetadata" 

# This is the action, we want to take 
# payload = {} # empty payload, all tables
# Example of setting a timeout
payload = { "timeout": {"timeUnit": "Seconds", "value": "60"}  }  # Setting a timeout for the REST call

try:
    response = client.post(uri,json= payload, lro_wait = True) 
    sync_status = json.loads(response.text)
    display(sync_status)
except Exception as e: print(e)